In [1]:
from newspaper import Article #newspaper has some nlp stuff, don't know if it is good enough

import spacy
from spacy import displacy
from collections import Counter
#! python3 -m spacy download en_core_web_lg
import en_core_web_lg
nlp = en_core_web_lg.load()

In [2]:
#https://towardsdatascience.com/scrape-and-summarize-news-articles-in-5-lines-of-python-code-175f0e5c7dfc
url = "https://thenewstack.io/dont-mess-with-the-master-working-with-branches-in-git-and-github/"
article = Article(url, language='en', fetch_images=False)

#download and parse the article
article.download()
article.parse()
#article.nlp() #Don't know if we need to parse

text = article.text

In [3]:
doc = nlp(text)
displacy.render(doc, jupyter=True, style='ent') 
labels = [x.label_ for x in doc.ents]
Counter(labels)

Counter({'ORG': 9,
         'CARDINAL': 10,
         'PRODUCT': 1,
         'ORDINAL': 2,
         'WORK_OF_ART': 9,
         'PERSON': 1})

In [4]:
sentences = [x for x in doc.sents]
print(sentences[21])

Master then is updated to contain all the new stuff.




In [5]:
items = [x.text for x in doc.ents]
Counter(items).most_common(3)

[('hello_octo', 7), ('GitHub', 4), ('hello_octo_world', 3)]

In [6]:
#! python3 -m nltk.downloader wordnet
#! python3 -m nltk.downloader omw
#! pip3 install spacy-wordnet

from spacy_wordnet.wordnet_annotator import WordnetAnnotator
nlp.add_pipe(WordnetAnnotator(nlp.lang), after='tagger')

In [7]:

for (item, count) in Counter(items).most_common(3):
    token = nlp(item)[0]
    print(token)
    print(token._.wordnet.synsets())
    #print(token._.wordnet.lemmas())
    #print(token._.wordnet.wordnet_domains())
    print()


hello_octo
[]

GitHub
[]

hello_octo_world
[]



In [8]:
url = "https://www.wsj.com/articles/titans-white-house-covid-outbreak-nfl-investigation-11601820641?mod=hp_lead_pos12"
article = Article(url, language='en', fetch_images=False)
article.download()
article.parse()
doc = nlp(article.text)
art_labels = [x.label_ for x in doc.ents]
art_ent = [x.text for x in doc.ents]
art_token = [nlp(item) for item in Counter(art_ent)]
print(Counter(labels))
print(doc.ents)

Counter({'CARDINAL': 10, 'ORG': 9, 'WORK_OF_ART': 9, 'ORDINAL': 2, 'PRODUCT': 1, 'PERSON': 1})
(Sept. 25, Tennessee, Shane Bowen, The next day, Minnesota, Vikings, A week later, Titans, Eighteen, two, Sunday)


In [11]:
text = "When Obama was president 60 million Americans got the H1N1 swine flu. Did you know that? How widely was it reported? And did you know that once tests indicated how widely it was that White House halted testing?"
doc = nlp(text)
labels = [x.label_ for x in doc.ents]
com_ent = [x.text for x in doc.ents]
com_token = [nlp(item) for item in Counter(com_ent)]
print(Counter(labels))
print(doc.ents)
displacy.render(doc, jupyter=True, style = 'ent') 

[Obama, 60 million, Americans, H1N1, White House]
Counter({'PERSON': 1, 'CARDINAL': 1, 'NORP': 1, 'DATE': 1, 'ORG': 1})
(Obama, 60 million, Americans, H1N1, White House)


In [19]:
for token in Counter(com_token):
    
        #initialize list of scores
        scores = []
        
        #for each article item
        for art_word in art_token:
            
            #add similarity score to list of scores
            scores += [art_word.similarity(token)]
        
        #output for each comment
        print(token)
        print(max(scores))
        print(art_token[scores.index(max(scores))])
        print(min(scores))
        print(art_token[scores.index(min(scores))])
        print(sum(scores)/len(art_token))
        print()

Obama
0.3977515187597841
Shane Bowen
0.05614126209834351
two
0.23230864680872176

60 million
0.44427766650130546
A week later
-0.03407282156621923
Shane Bowen
0.1790055059224109

Americans
0.3797265485709956
Minnesota
0.11950601665595888
Sept. 25
0.24824595198771987

H1N1
0.3209547548288165
A week later
0.05294525163005011
Vikings
0.15551054211463847

White House
0.5141195469872022
The next day
0.10691667633241796
Sept. 25
0.29313317229000085



In [ ]:
#https://medium.com/better-programming/the-beginners-guide-to-similarity-matching-using-spacy-782fc2922f7c
#another way to do it that might be better, takes the whole document